In [2]:
import pandas as pd

In [41]:
RIDERS = pd.read_csv("./docs/data/riders.csv")
RIDER_TEAMS = pd.read_csv("./docs/data/rider_teams.csv")
MANAGERS = pd.read_csv("./docs/data/managers.csv")
MANAGER_TEAMS = pd.read_csv("./docs/data/manager_teams.csv")
POINTS_SYSTEM = pd.read_csv("./docs/data/points_system.csv")